In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load

def train_svr_model(csv_path='data.csv', model_path='svr_model.joblib', scaler_path='scaler.joblib'):
    # Load dataset
    df = pd.read_csv(csv_path)

    # Pastikan semua kolom yang diperlukan ada
    required_columns = ['Waktu', 'TempOut', 'HumidityOut', 'TempAC','Temp']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Kolom '{col}' tidak ditemukan dalam dataset.")

    # Memilih fitur dan target
    X = df[['Waktu', 'TempOut', 'HumidityOut', 'TempAC']]
    y = df['Temp']

    # Split data untuk training dan testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scaling data
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train model SVR
    svr = SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)
    svr.fit(X_train_scaled, y_train)

    # Prediksi dan evaluasi
    y_pred = svr.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Print evaluasi model
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}")
    print(f"R² Score: {r2:.2f}")

    # Simpan model dan scaler
    dump(svr, model_path)
    dump(scaler, scaler_path)
    print(f"Model SVR disimpan di {model_path}")
    print(f"Scaler disimpan di {scaler_path}")

def predict_temp_ac(input_data, model_path='svr_model.joblib', scaler_path='scaler.joblib'):
    # Load model dan scaler
    model = load(model_path)
    scaler = load(scaler_path)

    # Konversi ke DataFrame dengan indeks 0
    input_df = pd.DataFrame([input_data], index=[0])

    # Scaling input
    input_scaled = scaler.transform(input_df)

    # Prediksi suhu AC
    predicted_temp_ac = model.predict(input_scaled)[0]
    return predicted_temp_ac

# Contoh training model
train_svr_model()

# Contoh prediksi dengan input
input_sample = {
    'Waktu': 1,  # Waktu dalam menit sejak tengah malam
    'TempOut': 1,  
    'HumidityOut': 1,  
    'TempAC': 1,  
}

predicted_temp = predict_temp_ac(input_sample)
print(f"Predicted TempAC: {predicted_temp:.2f}")


MAE: 0.05
MAPE: 0.00
R² Score: -0.07
Model SVR disimpan di svr_model.joblib
Scaler disimpan di scaler.joblib
Predicted TempAC: 30.95
